In [2]:
from pymongo import MongoClient

mongo_client = MongoClient()

database = mongo_client["graduation"]

pull_requests_collection = database["pull_requests"]

projects_collection = database["projects"]

commits_collection = database["commits"]

## Building the regex used to find the line number of commit chunks

In [3]:
import re

line_number_hunk_regex = "((^|\\n)\@\@\ -(\d+(|,\d+))\ \+(\d+(|,\d+))\ \@\@)"

compiled_regex = re.compile(line_number_hunk_regex, re.M)

projects = list(projects_collection.find({'travis_is_oldest_ci': True}))

In [6]:
for project in projects[:10]:
    prs = list(pull_requests_collection.find({'project_name': project["full_name"].split("/")[1],'project_owner': project["full_name"].split("/")[0]}))
    
    for pr in prs:
        commits = [commits_collection.find_one({'sha': commit}) for commit in pr["commits"]]
        
        for commit in commits:
            for file in commit["files"]:
                # If the patch of a file is empty we can't do anything
                # Empty patches can occur when for instance the status
                # is added or deleted, in which case a diff makes no
                # real sense. 
                if file["status"] == "modified" and \
                file["patch"] is not None:
                    matches = compiled_regex.findall(file["patch"])
                    
                    if len(matches) > 1:
                        #print("------")
                        for match in matches:
                            #print(match)
                            pass
                    elif len(matches) == 0:
                        print(commit["_id"])
                        print(file["patch"])
                    
                else:
                    #print(commit["_id"])
                    pass

In [69]:
import re

line_number_hunk_regex_raw = "((^|\\n)\@\@\ -(\d+(|,\d+))\ \+(\d+(|,\d+))\ \@\@)"

line_number_hunk_regex = re.compile(line_number_hunk_regex_raw)

'''
Given an item first checks whether it is a comment or 
commit. Then return the correct date field for the item. 
'''
def return_date_for_pr_or_commit(item):
    if "sha" in item:
        return item["date"]
    elif "original_commit_id" in item:
        return item["created_at"]
    else:
        raise ValueError("Ehm, this is not a code comment or commit")
        
        
'''
We do a sort of sweep line algorithm, where we sort commits and line 
comments on the time they have been posted. We go through them both 
chronologically, for a code comment we note its position, for a commit
we extract the position of the diff(s) and we note whether they 
intersect with any of the existing comments. If they intersect, we 
mark the comment as having been change inducing. 

Furthermore, we update the position of existing comments to account
for the lines that have been added or removed in diffs. 
'''
def process_pr(pr):
    
    comments = sorted([rc for rc in pr["review_comments"] if rc["in_reply_to_id"] is None], key=lambda comment : comment["created_at"])
    commits = list([commits_collection.find_one({'sha': commit, 'date': {'$exists': True}}) for commit in pr["commits"]])
    
    commits = list([cmt for cmt in commits if cmt is not None])
       
    commits.sort(key = lambda commit : commit["date"])
    
    if len(comments) == 0:
        return
        
    if len(commits)  == 0:
        print("\tOh misery! PR with number {} has no dated commits".format(pr["number"]))
        return
        
    merged_list = commits + comments
    
    merged_list.sort(key = return_date_for_pr_or_commit)
    
    placed_line_comments = []
    
    made_commits = {}
    
    for item in merged_list:
        # Only process newly placed comments, don't bother with comments
        # that are placed as a reaction
        if "original_commit_id" in item:
            # Add the comment on the moment it has been placed and
            # attribute it with the closest location in the new part of the 
            # diff
            placed_line_comments.append(item)

            lines = item["diff_hunk"].split("\n")
            
            curr_pos_in_old = 0
            curr_pos_in_new = 0
            
            lines_done = 0
            
            for line in lines:                
                # The below bit updates the lines based on the counter
                if line.startswith("@@"):
                    hunk_header_match = line_number_hunk_regex.match(line)
                    
                    curr_pos_in_old = int(hunk_header_match.group(3).split(",")[0])
                                        
                    curr_pos_in_new = int(hunk_header_match.group(5).split(",")[0])
                    
                elif line.startswith("-"):
                    curr_pos_in_old += 1
                elif line.startswith("+"):
                    curr_pos_in_new += 1
                elif line.startswith(" "):
                    curr_pos_in_old += 1
                    curr_pos_in_new += 1
                elif line.startswith("\\"):
                    pass
                else:
                    raise ValueError("Horror, there is diff panic")
                    
                lines_done += 1
                
                if item["original_position"] == lines_done:
                    # This is where the comment is --> record
                    # the position in the new file as the location 
                    # of this comment. 
                    #
                    # That way we can compare later commits to see
                    # if they came close to this comment
                    
                    item["eff_track_line"] = curr_pos_in_new
        
            
        elif "sha" in item:
            
            made_commits[item["sha"]] = item
            
            for file in item["files"]:
                # If the file has been modified we need to do two things.
                # Check if any omments were near any of the lines that have
                # been changed in this commit.
                # and if lines have been added or removed we need to update
                # the location of currently placed comments to reflect the
                # fact that they have changed by position by this commit
                if file["status"] == 'modified':
                    
                    for file_comment in [rc for rc in placed_line_comments if rc["path"] == file["filename"] and "eff_track_line" in rc]:
                        patch = file["patch"]
                        
                        if patch is None:
                            # This is a serious error condition!
                            return
                        
                        curr_pos_in_old = 0
                        curr_pos_in_new = 0
                        
                        lines_delta = 0
                        
                        for line in patch.split("\n"):
                            if line.startswith("@@"):
                                hunk_header_match = line_number_hunk_regex.match(line)

                                curr_pos_in_old = int(hunk_header_match.group(3).split(",")[0])

                                curr_pos_in_new = int(hunk_header_match.group(5).split(",")[0])
                                
                                # Positive for added lines
                                # negative for removed lines
                                lines_delta = curr_pos_in_new - curr_pos_in_old
                                
                                # At this point the diff passed the location of the comment. 
                                # We update the old position and we're done.
                                if curr_pos_in_old > file_comment["eff_track_line"]:
                                    file_comment["eff_track_line"] += lines_delta

                            elif line.startswith("-"):
                                curr_pos_in_old += 1
                            elif line.startswith("+"):
                                curr_pos_in_new += 1
                            elif line.startswith(" "):
                                curr_pos_in_old += 1
                                curr_pos_in_new += 1
                            elif line.startswith("\\"):
                                pass                                                 
                            else:
                                print(line)
                                raise ValueError("Horror, there is diff panic")

                            if file_comment["eff_track_line"] == curr_pos_in_old:
                                print("EFFECTIVE {}".format(file_comment["url"]))
                                
                            
                # If the file is deleted any comments in the 
                # file have induced some form of change
                elif file["status"] == "removed":
                    for file_comment in [rc for rc in placed_line_comments if rc["path"] == file["filename"]]:
                        """print("body: {}".format(file_comment["body"]))
                        print("sha: {}".format(item["sha"]))
                        print("filename: {}".format(file["filename"]))
                        print("url: {}".format(pr["html_url"]))
                        print("found effective comment")"""
                        #print("EFFECTIVE {}".format(file_comment["url"]))
                        placed_line_comments.remove(file_comment)
                    
        else:
            raise ValueError("Ehm this is not a code comment or commit")
    

for project in projects[:10]:
    prs = list(pull_requests_collection.find({'project_name': project["full_name"].split("/")[1],'project_owner': project["full_name"].split("/")[0]}))
    
    print("Analyzing {}".format(project["full_name"]))
    
    for pr in prs:
        process_pr(pr)
        

Analyzing Leaflet/Leaflet
EFFECTIVE https://api.github.com/repos/Leaflet/Leaflet/pulls/comments/169656808
EFFECTIVE https://api.github.com/repos/Leaflet/Leaflet/pulls/comments/162325627
EFFECTIVE https://api.github.com/repos/Leaflet/Leaflet/pulls/comments/162250560
EFFECTIVE https://api.github.com/repos/Leaflet/Leaflet/pulls/comments/162319393
EFFECTIVE https://api.github.com/repos/Leaflet/Leaflet/pulls/comments/162319393
EFFECTIVE https://api.github.com/repos/Leaflet/Leaflet/pulls/comments/155857929
EFFECTIVE https://api.github.com/repos/Leaflet/Leaflet/pulls/comments/152550733
EFFECTIVE https://api.github.com/repos/Leaflet/Leaflet/pulls/comments/147412404
EFFECTIVE https://api.github.com/repos/Leaflet/Leaflet/pulls/comments/144051320
EFFECTIVE https://api.github.com/repos/Leaflet/Leaflet/pulls/comments/145148959
EFFECTIVE https://api.github.com/repos/Leaflet/Leaflet/pulls/comments/144050698
EFFECTIVE https://api.github.com/repos/Leaflet/Leaflet/pulls/comments/144050698
EFFECTIVE http

EFFECTIVE https://api.github.com/repos/Leaflet/Leaflet/pulls/comments/65194665
EFFECTIVE https://api.github.com/repos/Leaflet/Leaflet/pulls/comments/65194941
EFFECTIVE https://api.github.com/repos/Leaflet/Leaflet/pulls/comments/65265369
EFFECTIVE https://api.github.com/repos/Leaflet/Leaflet/pulls/comments/65267038
EFFECTIVE https://api.github.com/repos/Leaflet/Leaflet/pulls/comments/65370502
EFFECTIVE https://api.github.com/repos/Leaflet/Leaflet/pulls/comments/65370992
EFFECTIVE https://api.github.com/repos/Leaflet/Leaflet/pulls/comments/65402311
EFFECTIVE https://api.github.com/repos/Leaflet/Leaflet/pulls/comments/65403684
EFFECTIVE https://api.github.com/repos/Leaflet/Leaflet/pulls/comments/65733722
EFFECTIVE https://api.github.com/repos/Leaflet/Leaflet/pulls/comments/60792305
EFFECTIVE https://api.github.com/repos/Leaflet/Leaflet/pulls/comments/60204429
EFFECTIVE https://api.github.com/repos/Leaflet/Leaflet/pulls/comments/59221672
EFFECTIVE https://api.github.com/repos/Leaflet/Leafl

EFFECTIVE https://api.github.com/repos/Leaflet/Leaflet/pulls/comments/22381900
EFFECTIVE https://api.github.com/repos/Leaflet/Leaflet/pulls/comments/22381900
EFFECTIVE https://api.github.com/repos/Leaflet/Leaflet/pulls/comments/21391451
EFFECTIVE https://api.github.com/repos/Leaflet/Leaflet/pulls/comments/21391451
EFFECTIVE https://api.github.com/repos/Leaflet/Leaflet/pulls/comments/21391451
EFFECTIVE https://api.github.com/repos/Leaflet/Leaflet/pulls/comments/21391451
EFFECTIVE https://api.github.com/repos/Leaflet/Leaflet/pulls/comments/23184196
EFFECTIVE https://api.github.com/repos/Leaflet/Leaflet/pulls/comments/23256381
EFFECTIVE https://api.github.com/repos/Leaflet/Leaflet/pulls/comments/20527546
EFFECTIVE https://api.github.com/repos/Leaflet/Leaflet/pulls/comments/20528916
EFFECTIVE https://api.github.com/repos/Leaflet/Leaflet/pulls/comments/14860203
EFFECTIVE https://api.github.com/repos/Leaflet/Leaflet/pulls/comments/10685473
EFFECTIVE https://api.github.com/repos/Leaflet/Leafl

Analyzing MRtrix3/mrtrix3
EFFECTIVE https://api.github.com/repos/MRtrix3/mrtrix3/pulls/comments/205733260
EFFECTIVE https://api.github.com/repos/MRtrix3/mrtrix3/pulls/comments/194026377
EFFECTIVE https://api.github.com/repos/MRtrix3/mrtrix3/pulls/comments/185734507
EFFECTIVE https://api.github.com/repos/MRtrix3/mrtrix3/pulls/comments/185736242
EFFECTIVE https://api.github.com/repos/MRtrix3/mrtrix3/pulls/comments/165949591
EFFECTIVE https://api.github.com/repos/MRtrix3/mrtrix3/pulls/comments/151883865
EFFECTIVE https://api.github.com/repos/MRtrix3/mrtrix3/pulls/comments/141082401
EFFECTIVE https://api.github.com/repos/MRtrix3/mrtrix3/pulls/comments/141082401
EFFECTIVE https://api.github.com/repos/MRtrix3/mrtrix3/pulls/comments/141082401
EFFECTIVE https://api.github.com/repos/MRtrix3/mrtrix3/pulls/comments/134691138
EFFECTIVE https://api.github.com/repos/MRtrix3/mrtrix3/pulls/comments/112650988
EFFECTIVE https://api.github.com/repos/MRtrix3/mrtrix3/pulls/comments/102726523
EFFECTIVE http

EFFECTIVE https://api.github.com/repos/MarkUsProject/Markus/pulls/comments/191847940
EFFECTIVE https://api.github.com/repos/MarkUsProject/Markus/pulls/comments/191847940
EFFECTIVE https://api.github.com/repos/MarkUsProject/Markus/pulls/comments/191818270
EFFECTIVE https://api.github.com/repos/MarkUsProject/Markus/pulls/comments/191818252
EFFECTIVE https://api.github.com/repos/MarkUsProject/Markus/pulls/comments/191818254
EFFECTIVE https://api.github.com/repos/MarkUsProject/Markus/pulls/comments/191818256
EFFECTIVE https://api.github.com/repos/MarkUsProject/Markus/pulls/comments/191818257
EFFECTIVE https://api.github.com/repos/MarkUsProject/Markus/pulls/comments/191818258
EFFECTIVE https://api.github.com/repos/MarkUsProject/Markus/pulls/comments/191831962
EFFECTIVE https://api.github.com/repos/MarkUsProject/Markus/pulls/comments/191833808
EFFECTIVE https://api.github.com/repos/MarkUsProject/Markus/pulls/comments/191822850
EFFECTIVE https://api.github.com/repos/MarkUsProject/Markus/pulls

...

EFFECTIVE https://api.github.com/repos/OCA/OpenUpgrade/pulls/comments/45918148
EFFECTIVE https://api.github.com/repos/OCA/OpenUpgrade/pulls/comments/45918213
EFFECTIVE https://api.github.com/repos/OCA/OpenUpgrade/pulls/comments/45978798
EFFECTIVE https://api.github.com/repos/OCA/OpenUpgrade/pulls/comments/45918097
EFFECTIVE https://api.github.com/repos/OCA/OpenUpgrade/pulls/comments/45918114
EFFECTIVE https://api.github.com/repos/OCA/OpenUpgrade/pulls/comments/45978689
EFFECTIVE https://api.github.com/repos/OCA/OpenUpgrade/pulls/comments/46033928
EFFECTIVE https://api.github.com/repos/OCA/OpenUpgrade/pulls/comments/46033990
EFFECTIVE https://api.github.com/repos/OCA/OpenUpgrade/pulls/comments/46034297
EFFECTIVE https://api.github.com/repos/OCA/OpenUpgrade/pulls/comments/46034393
EFFECTIVE https://api.github.com/repos/OCA/OpenUpgrade/pulls/comments/46034922
EFFECTIVE https://api.github.com/repos/OCA/OpenUpgrade/pulls/comments/46035066
EFFECTIVE https://api.github.com/repos/OCA/OpenUpgra

EFFECTIVE https://api.github.com/repos/OCA/OpenUpgrade/pulls/comments/31302534
EFFECTIVE https://api.github.com/repos/OCA/OpenUpgrade/pulls/comments/31302455
EFFECTIVE https://api.github.com/repos/OCA/OpenUpgrade/pulls/comments/26608618
EFFECTIVE https://api.github.com/repos/OCA/OpenUpgrade/pulls/comments/25570882
EFFECTIVE https://api.github.com/repos/OCA/OpenUpgrade/pulls/comments/25570918
EFFECTIVE https://api.github.com/repos/OCA/OpenUpgrade/pulls/comments/25570927
EFFECTIVE https://api.github.com/repos/OCA/OpenUpgrade/pulls/comments/25570882
EFFECTIVE https://api.github.com/repos/OCA/OpenUpgrade/pulls/comments/25570927
EFFECTIVE https://api.github.com/repos/OCA/OpenUpgrade/pulls/comments/25570882
EFFECTIVE https://api.github.com/repos/OCA/OpenUpgrade/pulls/comments/25570918
EFFECTIVE https://api.github.com/repos/OCA/OpenUpgrade/pulls/comments/25570927
EFFECTIVE https://api.github.com/repos/OCA/OpenUpgrade/pulls/comments/26655099
EFFECTIVE https://api.github.com/repos/OCA/OpenUpgra

Analyzing OCA/account-financial-tools
EFFECTIVE https://api.github.com/repos/OCA/account-financial-tools/pulls/comments/212795228
EFFECTIVE https://api.github.com/repos/OCA/account-financial-tools/pulls/comments/212897833
EFFECTIVE https://api.github.com/repos/OCA/account-financial-tools/pulls/comments/198769930
EFFECTIVE https://api.github.com/repos/OCA/account-financial-tools/pulls/comments/198770356
EFFECTIVE https://api.github.com/repos/OCA/account-financial-tools/pulls/comments/198770382
EFFECTIVE https://api.github.com/repos/OCA/account-financial-tools/pulls/comments/181286701
EFFECTIVE https://api.github.com/repos/OCA/account-financial-tools/pulls/comments/174648759
EFFECTIVE https://api.github.com/repos/OCA/account-financial-tools/pulls/comments/169955319
EFFECTIVE https://api.github.com/repos/OCA/account-financial-tools/pulls/comments/169955319
EFFECTIVE https://api.github.com/repos/OCA/account-financial-tools/pulls/comments/171569190
EFFECTIVE https://api.github.com/repos/OCA

EFFECTIVE https://api.github.com/repos/OCA/account-financial-tools/pulls/comments/94932859
EFFECTIVE https://api.github.com/repos/OCA/account-financial-tools/pulls/comments/94932955
EFFECTIVE https://api.github.com/repos/OCA/account-financial-tools/pulls/comments/94932972
EFFECTIVE https://api.github.com/repos/OCA/account-financial-tools/pulls/comments/94932955
EFFECTIVE https://api.github.com/repos/OCA/account-financial-tools/pulls/comments/94932955
EFFECTIVE https://api.github.com/repos/OCA/account-financial-tools/pulls/comments/91316542
EFFECTIVE https://api.github.com/repos/OCA/account-financial-tools/pulls/comments/91348106
EFFECTIVE https://api.github.com/repos/OCA/account-financial-tools/pulls/comments/95383188
EFFECTIVE https://api.github.com/repos/OCA/account-financial-tools/pulls/comments/95383257
EFFECTIVE https://api.github.com/repos/OCA/account-financial-tools/pulls/comments/82731441
EFFECTIVE https://api.github.com/repos/OCA/account-financial-tools/pulls/comments/82735913

EFFECTIVE https://api.github.com/repos/OCA/account-financial-tools/pulls/comments/26291819
EFFECTIVE https://api.github.com/repos/OCA/account-financial-tools/pulls/comments/26291819
EFFECTIVE https://api.github.com/repos/OCA/account-financial-tools/pulls/comments/26291819
EFFECTIVE https://api.github.com/repos/OCA/account-financial-tools/pulls/comments/26291819
EFFECTIVE https://api.github.com/repos/OCA/account-financial-tools/pulls/comments/25884154
EFFECTIVE https://api.github.com/repos/OCA/account-financial-tools/pulls/comments/25884154
EFFECTIVE https://api.github.com/repos/OCA/account-financial-tools/pulls/comments/25884282
EFFECTIVE https://api.github.com/repos/OCA/account-financial-tools/pulls/comments/25887463
EFFECTIVE https://api.github.com/repos/OCA/account-financial-tools/pulls/comments/25887620
EFFECTIVE https://api.github.com/repos/OCA/account-financial-tools/pulls/comments/24080931
EFFECTIVE https://api.github.com/repos/OCA/account-financial-tools/pulls/comments/24081781